In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
drive_path = "/content/drive/MyDrive/dat1"

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import tensorflow as tf
from tensorflow.keras.layers import  Conv2D, MaxPool2D, BatchNormalization, Activation, Input , Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model


In [ ]:
#Seeding the Environment
os.environ["PYTHONASHSEED"] = "42"
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#Hyperparameters
batch_size = 2
learning_rate = 1e-4
epochs = 40
height = 720
width = 1280

In [ ]:
#Paths
dataset_path = os.path.join(drive_path,"datasets", "augmented")
files_dir = os.path.join(drive_path, 'Colab Notebooks',"files", "augmented")
model_file= os.path.join(files_dir,"unet-original.h5")
log_file = os.path.join(files_dir,"log-original.csv")


In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir(files_dir)

In [ ]:
def conv_block(inputs, num_of_filters):
    x = Conv2D(num_of_filters,3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_of_filters,3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder(inputs, num_filters):
    x = conv_block(inputs,num_filters)
    p = MaxPool2D((2,2))(x)
    return x , p


def decoder(inputs, skip_connection, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x = Concatenate()([x,skip_connection])
    x = conv_block(x , num_filters)
    return x


def build_unet(input_shape):
    inputs = Input(input_shape)
    #encoder
    s1, p1 = encoder(inputs,64)
    s2, p2 = encoder(p1,128)
    s3, p3 = encoder(p2,256)
    s4, p4 = encoder(p3,512)


    #bridge
    b1 = conv_block(p4,1024)


    #decoder
    d1 = decoder(b1,s4,512)
    d2 = decoder(d1,s3,256)
    d3 = decoder(d2,s2,128)
    d4 = decoder(d3,s1,64)
    output = Conv2D(1,1,padding="same", activation = "sigmoid")(d4)


    model = Model(inputs, output, name="unet")
    return model


In [ ]:
#Data Pipeline

#load the dataset
def load_data(path):
    train_x = sorted(glob(os.path.join(path,"train","images","*")))
    train_y = sorted(glob(os.path.join(path,"train","masks","*")))

    valid_x = sorted(glob(os.path.join(path,"valid","images","*")))
    valid_y = sorted(glob(os.path.join(path,"valid","masks","*")))

    return (train_x,train_y) , (valid_x,valid_y)

#reading the image
def read_image(path):
    path = path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = x/255.0
    return x

#reading the mask
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x



In [ ]:
#tf data pipeline

def tf_parse(x,y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x, y

In [ ]:
def tf_dataset(x, y, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
(train_x,train_y) , (valid_x,valid_y) = load_data(dataset_path)

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [ ]:
input_shape = (height,width,3)

model = build_unet(input_shape)
model.summary()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 720, 1280, 3)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 720, 1280, 64)        1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 720, 1280, 64)        256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 720, 1280, 64)        0         ['batch_normalization[0][0]

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [ ]:
callbacks = [

    ModelCheckpoint(model_file,verbose=1,save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss",factor=0.1,patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss',patience=20, restore_best_weights= False)
]




In [ ]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs = epochs,
    callbacks=callbacks
)

Epoch 1/40
235/235 [==============================] - ETA: 0s - loss: 0.3504 - acc: 0.8702
Epoch 1: val_loss improved from inf to 0.66902, saving model to /content/drive/MyDrive/dat1/Colab Notebooks/files/augmented/unet-original.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


235/235 [==============================] - 548s 2s/step - loss: 0.3504 - acc: 0.8702 - val_loss: 0.6690 - val_acc: 0.6175 - lr: 1.0000e-04
Epoch 2/40
235/235 [==============================] - ETA: 0s - loss: 0.2470 - acc: 0.9025
Epoch 2: val_loss improved from 0.66902 to 0.26937, saving model to /content/drive/MyDrive/dat1/Colab Notebooks/files/augmented/unet-original.h5
235/235 [==============================] - 410s 2s/step - loss: 0.2470 - acc: 0.9025 - val_loss: 0.2694 - val_acc: 0.8982 - lr: 1.0000e-04
Epoch 3/40
235/235 [==============================] - ETA: 0s - loss: 0.2243 - acc: 0.9079
Epoch 3: val_loss improved from 0.26937 to 0.20074, saving model to /content/drive/MyDrive/dat1/Colab Notebooks/files/augmented/unet-original.h5
235/235 [==============================] - 409s 2s/step - loss: 0.2243 - acc: 0.9079 - val_loss: 0.2007 - val_acc: 0.9090 - lr: 1.0000e-04
Epoch 4/40
235/235 [==============================] - ETA: 0s - loss: 0.2191 - acc: 0.9054
Epoch 4: val_loss di